**Notes : **

1 - custom prediction python script should be provided (scoring.py)

2 - will check if artefacts can be installed to notebook

In [45]:
# import required libraries
import os
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
    ProbeSettings,
    Environment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

from azureml.core import (
    Experiment,
    Workspace,
    Run
)

from azure.identity import DefaultAzureCredential

In [46]:
name = "nnunet-devops-test-2"
# input training step id
latest_run_id = "d134788d-094b-4cfa-b38b-c2eced07ba53" #"d6c83f1e-97a7-48a1-b4db-a308de7bd9e2" # gets the latest if empty

In [47]:
workspace = Workspace.from_config()
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


In [48]:
# get experiment
experiment = Experiment(workspace, name)
# get experiment's latest run id
if latest_run_id:
    latest_run = Run(experiment, latest_run_id) 
else:
    latest_run = next(experiment.get_runs())

    
print(latest_run)
print(type(latest_run))


Run(Experiment: nnunet-devops-test-2,
Id: d134788d-094b-4cfa-b38b-c2eced07ba53,
Type: azureml.StepRun,
Status: Completed)
<class 'azureml.core.run.Run'>


In [49]:
model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnUNet/"

In [50]:
# model registration procedure
model = Model(path=model_path,
                name="iamnewtestmodel",
                description="description-here",
                type=AssetTypes.CUSTOM_MODEL)
registered_model = ml_client.models.create_or_update(model)

In [51]:
# Creating a unique endpoint name with current datetime to avoid conflicts
online_endpoint_name = "nnunet-model-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

nnunet-model-07141653760676


In [52]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://nnunet-model-07141653760676.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://nnunet-model-07141653760676.eastus2.inference.ml.azure.com/swagger.json', 'name': 'nnunet-model-07141653760676', 'description': '', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourcegroups/azureml-test/providers/microsoft.machinelearningservices/workspaces/nnunet/onlineendpoints/nnunet-model-07141653760676', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:438978f1-ead1-47c3-ad8e-c77be361c727:b35f92c3-9fb7-4054-80a6-3af70229d13d?api-version=2022-02-01-preview'}, 'id': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourceGroups/azureml-test/providers/Micros

In [53]:
#docker_env.docker.base_image = "nnunet.azurecr.io/training_env" # example : "fastdotai/fastai2:latest"
#docker_env.python.user_managed_dependencies = True

deployment = ManagedOnlineDeployment(
    name=name,
    environment=Environment(image="nnunet.azurecr.io/prediction_env:latest"),
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS1_v2", #"Standard_DS2_v2",
    code_configuration=CodeConfiguration(code="", scoring_script="src/score.py"),
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [54]:
ml_client.online_deployments.begin_create_or_update(deployment).result()


Check: endpoint nnunet-model-07141653760676 exists
Uploading avaj1870 (0.58 MBs): 100%|██████████| 583512/583512 [00:00<00:00, 804867.80it/s] 


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ManagedOnlineDeployment({'private_network_connection': False, 'data_collector': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'nnunet-model-07141653760676', 'type': 'Managed', 'name': 'nnunet-devops-test-2', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/od:438978f1-ead1-47c3-ad8e-c77be361c727:134e9904-b06b-42f5-959f-5efe28777418?api-version=2022-02-01-preview'}, 'id': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourceGroups/azureml-test/providers/Microsoft.MachineLearningServices/workspaces/nnunet/onlineEndpoints/nnunet-model-07141653760676/deployments/nnunet-devops-test-2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/avaj18701/code/Users/avaj1870', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object 

In [55]:
# update deployment to take 100% traffic
endpoint.traffic = {name: 0}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://nnunet-model-07141653760676.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://nnunet-model-07141653760676.eastus2.inference.ml.azure.com/swagger.json', 'name': 'nnunet-model-07141653760676', 'description': '', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourcegroups/azureml-test/providers/microsoft.machinelearningservices/workspaces/nnunet/onlineendpoints/nnunet-model-07141653760676', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:438978f1-ead1-47c3-ad8e-c77be361c727:c69c855d-3bfb-45a7-bc34-65fac277d5a8?api-version=2022-02-01-preview'}, 'id': '/subscriptions/e5fb5c5f-cb5c-42c8-86b6-cf4fd5aac11b/resourceGroups/azureml-test/providers/Micros